In [31]:
# =====================================================
# BASELINE MODEL TRAINING (STYLE METRICS ONLY)
# =====================================================

import sys
from pathlib import Path

PROJECT_ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))


# =========================
# IMPORTS
# =========================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

from tqdm import tqdm

from src.features.text_metrics import TextMetricCalculator
from src.config.config import config


# =========================
# LOAD DATA
# =========================

df = pd.read_csv(
    PROJECT_ROOT / "data" / "processed" / "balanced_length_filtered_dataset.csv"
)

print("Dataset shape:", df.shape)
print(df["generated"].value_counts())


# =========================
# BUILD METRICS DATAFRAME
# =========================

def build_metrics_dataframe(
    df: pd.DataFrame,
    text_col: str = "text",
    label_col: str = "generated",
) -> pd.DataFrame:
    rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Extracting text metrics"):
        calculator = TextMetricCalculator(row[text_col])
        metrics = calculator.all_metrics

        metrics_dict = vars(metrics)
        metrics_dict[label_col] = row[label_col]

        rows.append(metrics_dict)

    return pd.DataFrame(rows)


metrics_df = build_metrics_dataframe(df)

print("\nMetrics DataFrame shape:", metrics_df.shape)
print(metrics_df.head())


# =========================
# FEATURE / TARGET SPLIT
# =========================

TARGET_COL = "generated"

X = metrics_df.drop(columns=[TARGET_COL])
y = metrics_df[TARGET_COL]

print("\nFeature matrix shape:", X.shape)
print("Target distribution:\n", y.value_counts())


# =========================
# TRAIN / TEST SPLIT
# =========================

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y,
)

print("\nTrain size:", X_train.shape)
print("Test size:", X_test.shape)


# =========================
# MODEL PIPELINE
# =========================

pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        (
            "classifier",
            LogisticRegression(
                max_iter=1000,
                class_weight="balanced",
                random_state=42,
            ),
        ),
    ]
)


# =========================
# TRAIN
# =========================

pipeline.fit(X_train, y_train)


# =========================
# EVALUATION
# =========================

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n===== MODEL PERFORMANCE =====")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=["Human-written", "AI-generated"]))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))


# =========================
# FEATURE IMPORTANCE
# =========================

feature_importance = pd.Series(
    pipeline.named_steps["classifier"].coef_[0],
    index=X.columns,
).sort_values(key=np.abs, ascending=False)

print("\n===== FEATURE IMPORTANCE (LogReg coefficients) =====")
print(feature_importance)


Dataset shape: (70000, 3)
generated
0.0    35000
1.0    35000
Name: count, dtype: int64


Extracting text metrics: 100%|██████████| 70000/70000 [00:16<00:00, 4367.19it/s]



Metrics DataFrame shape: (70000, 15)
   word_count  text_length  sentence_count  avg_sentence_length  \
0         312         1452              16             3.586538   
1         346         2051              18             4.809249   
2         324         1825              22             4.320988   
3         354         1832              21             4.084746   
4         319         1597              17             3.937304   

   avg_word_length  vocab_size  vocab_richness  repetition_ratio  \
0         4.653846         125        0.400641          0.599359   
1         5.927746         187        0.540462          0.459538   
2         5.632716         157        0.484568          0.515432   
3         5.175141         146        0.412429          0.587571   
4         5.006270         127        0.398119          0.601881   

   punctuation_ratio  exclamation_ratio  question_ratio  uppercase_word_ratio  \
0           0.015152           0.001377        0.000000              

In [32]:
# =====================================================
# HUMAN vs AI TEXT CLASSIFIER
# TF-IDF + STYLOMETRIC FEATURES (COMBINED MODEL)
# =====================================================

import sys
from pathlib import Path
import joblib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

from tqdm import tqdm

# ---- project imports ----
sys.path.insert(0, str(PROJECT_ROOT))

from src.features.text_metrics import TextMetricCalculator
from src.config.config import config


# =====================================================
# PATHS
# =====================================================

DATA_PATH = PROJECT_ROOT / "data" / "processed" / "balanced_length_filtered_dataset.csv"
MODEL_PATH = PROJECT_ROOT / "models" / "combined_tfidf_style_model.joblib"
MODEL_PATH.parent.mkdir(exist_ok=True)


# =====================================================
# LOAD DATA
# =====================================================

df = pd.read_csv(DATA_PATH)
df = df.dropna(subset=["text", "generated"])

df["generated"] = df["generated"].astype(int)

print("Dataset shape:", df.shape)
print(df["generated"].value_counts())


# =====================================================
# BUILD STYLE METRICS
# =====================================================

def build_metrics_dataframe(
    df: pd.DataFrame,
    text_col: str = "text",
) -> pd.DataFrame:
    rows = []

    for text in tqdm(df[text_col], desc="Extracting text metrics"):
        metrics = TextMetricCalculator(text).all_metrics
        rows.append(vars(metrics))

    return pd.DataFrame(rows)


style_df = build_metrics_dataframe(df)

STYLE_FEATURES = style_df.columns.tolist()

print("Style features:", STYLE_FEATURES)


# =====================================================
# FINAL DATAFRAME
# =====================================================

full_df = pd.concat(
    [
        df[["text", "generated"]].reset_index(drop=True),
        style_df.reset_index(drop=True),
    ],
    axis=1,
)

X = full_df.drop(columns=["generated"])
y = full_df["generated"]


# =====================================================
# TRAIN / TEST SPLIT
# =====================================================

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y,
)


# =====================================================
# COLUMN TRANSFORMER
# =====================================================

preprocessor = ColumnTransformer(
    transformers=[
        (
            "tfidf",
            TfidfVectorizer(
                lowercase=True,
                stop_words="english",
                max_features=20000,
                ngram_range=(1, 2),
                min_df=5,
                max_df=0.9,
            ),
            "text",
        ),
        (
            "style",
            StandardScaler(),
            STYLE_FEATURES,
        ),
    ],
    remainder="drop",
)


# =====================================================
# PIPELINE
# =====================================================

pipeline = Pipeline(
    steps=[
        ("features", preprocessor),
        (
            "classifier",
            LogisticRegression(
                max_iter=1000,
                class_weight="balanced",
                n_jobs=-1,
                random_state=42,
            ),
        ),
    ]
)


# =====================================================
# TRAINING
# =====================================================

print("\nTraining combined model...")
pipeline.fit(X_train, y_train)
print("Training finished.")


# =====================================================
# EVALUATION
# =====================================================

y_pred = pipeline.predict(X_test)

print("\n===== MODEL PERFORMANCE =====")
print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall   : {recall_score(y_test, y_pred):.4f}")
print(f"F1-score : {f1_score(y_test, y_pred):.4f}")

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=["Human", "AI"]))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))


# =====================================================
# SAVE MODEL
# =====================================================

joblib.dump(pipeline, MODEL_PATH)
print(f"\nModel saved to: {MODEL_PATH}")


# =====================================================
# INFERENCE
# =====================================================

def predict_text(text: str) -> dict:
    model = joblib.load(MODEL_PATH)

    style_metrics = vars(TextMetricCalculator(text).all_metrics)
    sample_df = pd.DataFrame(
        [{**{"text": text}, **style_metrics}]
    )

    proba = model.predict_proba(sample_df)[0]
    pred = model.predict(sample_df)[0]

    return {
        "prediction": "AI-generated" if pred == 1 else "Human-written",
        "probability_ai": float(proba[1]),
        "probability_human": float(proba[0]),
    }


# =====================================================
# EXAMPLE
# =====================================================


Dataset shape: (70000, 3)
generated
0    35000
1    35000
Name: count, dtype: int64


Extracting text metrics: 100%|██████████| 70000/70000 [00:13<00:00, 5218.68it/s]


Style features: ['word_count', 'text_length', 'sentence_count', 'avg_sentence_length', 'avg_word_length', 'vocab_size', 'vocab_richness', 'repetition_ratio', 'punctuation_ratio', 'exclamation_ratio', 'question_ratio', 'uppercase_word_ratio', 'entropy', 'stopwords_ratio']

Training combined model...


d:\Arsenij1\human-vs-ai-text-classifier\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Training finished.

===== MODEL PERFORMANCE =====
Accuracy : 0.9915
Precision: 0.9935
Recall   : 0.9896
F1-score : 0.9915

Classification report:
              precision    recall  f1-score   support

       Human       0.99      0.99      0.99      8750
          AI       0.99      0.99      0.99      8750

    accuracy                           0.99     17500
   macro avg       0.99      0.99      0.99     17500
weighted avg       0.99      0.99      0.99     17500

Confusion matrix:
[[8693   57]
 [  91 8659]]

Model saved to: D:\Arsenij1\human-vs-ai-text-classifier\models\combined_tfidf_style_model.joblib


In [34]:
if __name__ == "__main__":
    sample_text = """
    123
    """

    result = predict_text(sample_text)
    print("\n===== SAMPLE PREDICTION =====")
    print(result)


===== SAMPLE PREDICTION =====
{'prediction': 'Human-written', 'probability_ai': 5.5622566591327496e-05, 'probability_human': 0.9999443774334087}


In [ ]:
df1 = pd.read_csv(PROJECT_ROOT / "data" / "raw" / "AI Generated Essays Dataset.csv")

all = len(df1)
correct = 0

for _, row in df1.iterrows():
    text, ai = row['text'], row['generated']
    prediction = predict_text(text)['probability_ai']

    if prediction > 0.7:
        prediction = 1
    else:
        prediction = 0

    if prediction == ai:
        correct += 1

accuracy = correct / all
print(f"Accuracy on AI Generated Essays Dataset: {accuracy:.4f}")

    

Accuracy on AI Generated Essays Dataset: 0.0568
